Здесь нам предлагают решить известную задачу про Многорукого бандита и намекают на использование Томпсоновского сэмплирования тем, что пишут, что вероятности выигрыша у каждого автомата выбираются из бета-распределения.
Собственно, ниже представлена реализация этого метода решения задачи.
Env - среда, в которой происходит взаимодействие с автоматами, в которой мы выбираем автомат, получаем награду и обновляем состояние нашего агента.
BaseSampler - базовый класс агента, ThompsonSampler - агент на основе Томпсоновского сэмплирования.
Более подробно можно почитать [тут](https://habr.com/ru/articles/689364/). Еще более подробно в книгах, которые указал автор статьи, я лично дальше общих познаний пока не продвинулся.

In [ ]:
import numpy as np


class Environment:
    def __init__(self, variants, n_trials):
        self.variants = variants
        self.n_trials = n_trials
        self.total_reward = 0
        self.n_k = len(variants)
        self.shape = (self.n_k, n_trials)

    def run(self, agent):
        for i in range(self.n_trials):
            # делаем выбор
            x_chosen = agent.choose_k()
            # получаем награду
            reward = int(input())
            # запоминаем награду
            agent.reward = reward
            # агент обновляет параметры
            agent.update()
            self.total_reward += reward

        return self.total_reward


class BaseSampler:
    def __init__(self, env, n_samples=None, n_learning=None, e=0.05, alpha=1, beta=1):
        self.env = env
        self.shape = (env.n_k, n_samples)
        self.variants = env.variants
        self.n_trials = env.n_trials
        self.ad_i = np.zeros(env.n_trials)
        self.r_i = np.zeros(env.n_trials)
        self.thetas = np.zeros(self.n_trials)
        self.regret_i = np.zeros(env.n_trials)
        self.thetaregret = np.zeros(self.n_trials)

        self.a = np.full(env.n_k, alpha)
        self.b = np.full(env.n_k, beta)
        self.theta = np.zeros(env.n_k)
        self.data = None
        self.reward = 0
        self.total_reward = 0
        self.k = 0
        self.i = 0

        self.n_samples = n_samples
        self.n_learning = n_learning
        self.e = e
        self.ep = np.random.uniform(0, 1, size=env.n_trials)
        self.exploit = (1 - e)


class ThompsonSampler(BaseSampler):

    def __init__(self, env):
        super().__init__(env)

    def choose_k(self):
        self.theta = np.random.beta(self.a, self.b)
        # выбираем автомат
        self.k = self.variants[np.argmax(self.theta)]
        # выводим номер автомата
        print(self.k+1)

        return self.k

    def update(self):
        self.a[self.k] += self.reward
        self.b[self.k] += 1 - self.reward

        self.thetas[self.i] = self.theta[self.k]
        self.thetaregret[self.i] = np.max(self.thetas) - self.theta[self.k]

        self.ad_i[self.i] = self.k
        self.r_i[self.i] = self.reward
        self.i += 1


while True:
    n, m = map(int, input().split())
    if n == 0 and m == 0:
        break
    a, b = map(float, input().split())
    machines = [i for i in range(m)]
    en = Environment(machines, n_trials=n)
    sampler = ThompsonSampler(env=en)
    en.run(agent=sampler)
